hist, new, total, new/hist で特徴量作成

https://www.kaggle.com/code/raddar/target-true-meaning-revealed

2**target

amountを調整

np.round(df['purchase_amount'] / 0.00150265118 + 497.06)

In [2]:
import warnings
import time
import sys
import datetime
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

%matplotlib inline

In [1]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def preprocessing(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    for col in ['category_3']:
        df[col] = df[col].map({'A':1, 'B':2,'C':3})
    return df

def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    return df

In [20]:
# データの読み込み
path_new = '../../../../data/raw/new_merchant_transactions.csv'
path_hist = '../../../../data/raw/historical_transactions.csv'
path_train = '../../../../data/raw/train.csv'
path_test = '../../../../data/raw/test.csv'

new = pd.read_csv(path_new, parse_dates=['purchase_date'])
hist = pd.read_csv(path_hist ,parse_dates=['purchase_date'])

new = preprocessing(new)
hist = preprocessing(hist)

new = reduce_mem_usage(new)
hist = reduce_mem_usage(hist)

train = pd.read_csv(path_train, parse_dates=['first_active_month'])
test = pd.read_csv(path_test, parse_dates=['first_active_month'])

target = train['target']
#del train['target']

Mem. usage decreased to 76.76 Mb (63.4% reduction)
Mem. usage decreased to 1193.84 Mb (61.6% reduction)


In [21]:
train = pd.read_csv(path_train, parse_dates=['first_active_month'])
test = pd.read_csv(path_test, parse_dates=['first_active_month'])

## train, test 処理

In [22]:
train['target'] = 2**train['target']

In [23]:
test_null_id = test[test.first_active_month.isna()].card_id.iloc[-1]
null_month = hist[hist.card_id == test_null_id].purchase_date.dt.strftime("%Y-%m-01").min()
test.fillna({'first_active_month':null_month},inplace=True)

In [24]:
for df in [test, train]:
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days

In [25]:
for df in [test,train]:
    df['first_active_month']  = (df['first_active_month'] - df['first_active_month'].min()).dt.days

## 特徴量作成

In [34]:
for df in [hist, new]:
    df['purchase_amount'] = np.round(df['purchase_amount'] / 0.00150265118 + 497.06)

In [35]:
hist

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount2
0,1,C_ID_4e6213e9bc,88,0,0,1.0,80,M_ID_e020e9b302,-8,29.0,2017-06-25 15:33:07,1.0,16,37,29.0
1,1,C_ID_4e6213e9bc,88,0,0,1.0,367,M_ID_86ec983688,-7,9.0,2017-07-15 12:10:45,1.0,16,16,9.0
2,1,C_ID_4e6213e9bc,88,0,0,1.0,80,M_ID_979ed661fc,-6,18.0,2017-08-09 22:04:29,1.0,16,37,18.0
3,1,C_ID_4e6213e9bc,88,0,0,1.0,560,M_ID_e6d5ae8ea6,-5,8.0,2017-09-02 10:06:26,1.0,16,34,8.0
4,1,C_ID_4e6213e9bc,88,0,0,1.0,80,M_ID_e020e9b302,-11,16.0,2017-03-10 01:14:19,1.0,16,37,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112356,1,C_ID_2863d2fa95,-1,1,1,2.0,360,M_ID_edd92b6720,-1,76.0,2017-01-20 08:52:04,NaN,-1,34,76.0
29112357,1,C_ID_2863d2fa95,-1,1,1,2.0,360,M_ID_edd92b6720,0,76.0,2017-02-20 04:40:50,NaN,-1,34,76.0
29112358,1,C_ID_5c240d6e3c,3,0,0,1.0,278,M_ID_9cdcfe8673,0,59.0,2017-12-26 18:37:51,1.0,16,37,59.0
29112359,1,C_ID_5c240d6e3c,331,0,0,1.0,514,M_ID_1a75f94f92,-1,1010.0,2017-11-24 14:18:15,1.0,16,9,1010.0


In [36]:
for df in [hist, new]:
    df['category_2'].fillna(1,inplace=True) # 1が最頻値
    df['category_3'].fillna(1,inplace=True) # 1が最頻値
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True) # 最頻値

In [37]:
# 承認フラグ　Yes,Noの数と比
# histのみ（newはYesのみのため）
authorized_flag = hist[['card_id','authorized_flag']].groupby(['card_id','authorized_flag']).agg(count=('card_id','count')).reset_index()

# カードIDごとの合計を計算
total_counts = authorized_flag.groupby('card_id')['count'].sum().reset_index()
total_counts = total_counts.rename(columns={'count': 'total_count'})

# 元のデータと合計を結合
authorized_flag = authorized_flag.merge(total_counts, on='card_id')

# 割合を計算
authorized_flag['ratio'] = authorized_flag['count'] / authorized_flag['total_count']
#hist[['card_id','authorized_flag']].groupby(['card_id']).agg(count=('authorized_flag','mean')).reset_index()

# 結果を整形
authorized_ratio = authorized_flag.pivot(index='card_id', columns='authorized_flag', values=['ratio','count']).reset_index().rename(columns={0: 'No', 1: 'Yes'})
authorized_ratio.columns = ['_'.join(col).strip() for col in authorized_ratio.columns.values]
authorized_ratio.rename(columns={'card_id_': 'card_id'},inplace=True)

In [38]:
hist_id_u = hist.groupby('card_id',as_index=False)\
    .agg(
            frequency = ('card_id','count'),
            amount_total = ('purchase_amount','sum'),
            amount_mean = ('purchase_amount','mean'),
            amount_max = ('purchase_amount','max'),
            amount_min = ('purchase_amount','min'),
            amount_median = ('purchase_amount','median'),
            merchant_category_nu = ('merchant_category_id','nunique'),
            merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
            city_nu = ('city_id','nunique'),
            city_mode = ('city_id',lambda x: x.mode()[0]),
            state_nu = ('state_id','nunique'),
            state_mode = ('state_id',lambda x: x.mode()[0]),
            date_min  = ('purchase_date','min'),
            date_max  = ('purchase_date','max'),
            installments_mean = ('installments','mean'),
            installments_max = ('installments','max'),
            installments_min = ('installments','min'),
            installments_median = ('installments','median'),
            installments_sum = ('installments','sum'),
            month_lag_mean = ('month_lag','mean'),
            month_lag_max = ('month_lag','max'),
            month_lag_min = ('month_lag','min'),
            month_lag_median = ('month_lag','median'),
            category_1_mod = ('category_1',lambda x: x.mode()[0]),
            category_2_mod = ('category_2',lambda x: x.mode()[0]),
            category_3_mod = ('category_3',lambda x: x.mode()[0]),
        )

In [39]:
hist_id_u = hist_id_u.merge(authorized_ratio,on='card_id')

In [40]:
hist_id_u['duration'] = (hist_id_u['date_max'] - hist_id_u['date_min']).dt.days
hist_id_u.date_max = (hist_id_u.date_max - hist_id_u.date_min.min()).dt.days
hist_id_u.date_min = (hist_id_u.date_min - hist_id_u.date_min.min()).dt.days

hist_id_u.merchant_category_mode = pd.to_numeric(hist_id_u.merchant_category_mode, errors='coerce')

hist_id_u.columns = ['hist_' + col if col != 'card_id' else col for col in hist_id_u.columns]

In [41]:
new_id_u = new.groupby('card_id',as_index=False)\
    .agg(
            frequency = ('card_id','count'),
            amount_total = ('purchase_amount','sum'),
            amount_mean = ('purchase_amount','mean'),
            amount_max = ('purchase_amount','max'),
            amount_min = ('purchase_amount','min'),
            amount_median = ('purchase_amount','median'),
            merchant_category_nu = ('merchant_category_id','nunique'),
            merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
            city_nu = ('city_id','nunique'),
            city_mode = ('city_id',lambda x: x.mode()[0]),
            state_nu = ('state_id','nunique'),
            state_mode = ('state_id',lambda x: x.mode()[0]),
            date_min  = ('purchase_date','min'),
            date_max  = ('purchase_date','max'),
            installments_mean = ('installments','mean'),
            installments_max = ('installments','max'),
            installments_min = ('installments','min'),
            installments_median = ('installments','median'),
            installments_sum = ('installments','sum'),
            month_lag_mean = ('month_lag','mean'),
            month_lag_max = ('month_lag','max'),
            month_lag_min = ('month_lag','min'),
            month_lag_median = ('month_lag','median'),
            category_1_mod = ('category_1',lambda x: x.mode()[0]),
            category_2_mod = ('category_2',lambda x: x.mode()[0]),
            category_3_mod = ('category_3',lambda x: x.mode()[0]),
        )

In [42]:
new_id_u['duration'] = (new_id_u['date_max'] - new_id_u['date_min']).dt.days
new_id_u.date_max = (new_id_u.date_max - new_id_u.date_min.min()).dt.days
new_id_u.date_min = (new_id_u.date_min - new_id_u.date_min.min()).dt.days

new_id_u.merchant_category_mode = pd.to_numeric(new_id_u.merchant_category_mode, errors='coerce')

new_id_u.columns = ['new_' + col if col != 'card_id' else col for col in new_id_u.columns]

In [43]:
total = pd.concat([hist,new],axis=0).reset_index(drop=True)

In [44]:
total_id_u = total.groupby('card_id',as_index=False)\
    .agg(
            frequency = ('card_id','count'),
            amount_total = ('purchase_amount','sum'),
            amount_mean = ('purchase_amount','mean'),
            amount_max = ('purchase_amount','max'),
            amount_min = ('purchase_amount','min'),
            amount_median = ('purchase_amount','median'),
            merchant_category_nu = ('merchant_category_id','nunique'),
            merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
            city_nu = ('city_id','nunique'),
            city_mode = ('city_id',lambda x: x.mode()[0]),
            state_nu = ('state_id','nunique'),
            state_mode = ('state_id',lambda x: x.mode()[0]),
            date_min  = ('purchase_date','min'),
            date_max  = ('purchase_date','max'),
            installments_mean = ('installments','mean'),
            installments_max = ('installments','max'),
            installments_min = ('installments','min'),
            installments_median = ('installments','median'),
            installments_sum = ('installments','sum'),
            month_lag_mean = ('month_lag','mean'),
            month_lag_max = ('month_lag','max'),
            month_lag_min = ('month_lag','min'),
            month_lag_median = ('month_lag','median'),
            category_1_mod = ('category_1',lambda x: x.mode()[0]),
            category_2_mod = ('category_2',lambda x: x.mode()[0]),
            category_3_mod = ('category_3',lambda x: x.mode()[0]),
        )

In [45]:
total_id_u['duration'] = (total_id_u['date_max'] - total_id_u['date_min']).dt.days
total_id_u.date_max = (total_id_u.date_max - total_id_u.date_min.min()).dt.days
total_id_u.date_min = (total_id_u.date_min - total_id_u.date_min.min()).dt.days

total_id_u.merchant_category_mode = pd.to_numeric(total_id_u.merchant_category_mode, errors='coerce')

total_id_u.columns = ['total_' + col if col != 'card_id' else col for col in total_id_u.columns]

In [46]:
train = train.merge(hist_id_u, on='card_id',how='left')
test = test.merge(hist_id_u, on='card_id',how='left')

train = train.merge(new_id_u, on='card_id',how='left')
test = test.merge(new_id_u, on='card_id',how='left')

train = train.merge(total_id_u, on='card_id',how='left')
test = test.merge(total_id_u, on='card_id',how='left')

In [47]:
for df in [train, test]:
    # df['hist_count_No'].fillna(0,inplace=True)
    # df['hist_ratio_No'].fillna(0,inplace=True)
    df.fillna(0, inplace=True)

In [48]:
for df in [train, test]:
    df['ratio_amount_total'] = df['new_amount_total'] / df['hist_amount_total']
    df['ratio_amount_mean'] = df['new_amount_mean'] / df['hist_amount_mean']
    df['ratio_merchant_category_nu'] = df['new_merchant_category_nu'] / df['hist_merchant_category_nu']
    #df['ratio_installments_mean'] = df['new_installments_mean'] / df['hist_installments_mean']
    #df['ratio_installments_sum'] = df['new_installments_sum'] / df['hist_installments_sum']

In [49]:
path_train_prepro = '../../../../data/processed/train_processed_23.csv'
path_test_preprp = '../../../../data/processed/test_processed_23.csv'

train.to_csv(path_train_prepro,index=False)
test.to_csv(path_test_preprp,index=False)